# importing libraries

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data import

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/weather.csv")
df

,rownames,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,1,EWR,2013,1,1,1,39.02,26.06,59.37,270.0,10.35702,NaN,0.0,1012.0,10.0,2013-01-01T06:00:00Z
1,2,EWR,2013,1,1,2,39.02,26.96,61.63,250.0,8.05546,NaN,0.0,1012.3,10.0,2013-01-01T07:00:00Z
2,3,EWR,2013,1,1,3,39.02,28.04,64.43,240.0,11.50780,NaN,0.0,1012.5,10.0,2013-01-01T08:00:00Z
3,4,EWR,2013,1,1,4,39.92,28.04,62.21,250.0,12.65858,NaN,0.0,1012.2,10.0,2013-01-01T09:00:00Z
4,5,EWR,2013,1,1,5,39.02,28.04,64.43,260.0,12.65858,NaN,0.0,1011.9,10.0,2013-01-01T10:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26110,26111,LGA,2013,12,30,14,35.96,19.94,51.78,340.0,13.80936,21.86482,0.0,1017.1,10.0,2013-12-30T19:00:00Z
26111,26112,LGA,2013,12,30,15,33.98,17.06,49.51,330.0,17.26170,21.86482,0.0,1018.8,10.0,2013-12-30T20:00:00Z
26112,26113,LGA,2013,12,30,16,32.00,15.08,49.19,340.0,14.96014,23.01560,0.0,1019.5,10.0,2013-12-30T21:00:00Z
26113,26114,LGA,2013,12,30,17,30.92,12.92,46.74,320.0,17.26170,NaN,0.0,1019.9,10.0,2013-12-30T22:00:00Z


# Data processing and cleaning

In [ ]:
# Drop unused comments
df.drop(columns=["rownames", "origin", "wind_dir", "wind_gust", "visib","year", "day", "dewp", "precip", "time_hour"],axis=1, inplace=True)
df

,year,month,day,hour,temp,dewp,humid,wind_speed,precip,pressure
0,2013,1,1,1,39.02,26.06,59.37,10.35702,0.0,1012.0
1,2013,1,1,2,39.02,26.96,61.63,8.05546,0.0,1012.3
2,2013,1,1,3,39.02,28.04,64.43,11.50780,0.0,1012.5
3,2013,1,1,4,39.92,28.04,62.21,12.65858,0.0,1012.2
4,2013,1,1,5,39.02,28.04,64.43,12.65858,0.0,1011.9
...,...,...,...,...,...,...,...,...,...,...
26110,2013,12,30,14,35.96,19.94,51.78,13.80936,0.0,1017.1
26111,2013,12,30,15,33.98,17.06,49.51,17.26170,0.0,1018.8
26112,2013,12,30,16,32.00,15.08,49.19,14.96014,0.0,1019.5
26113,2013,12,30,17,30.92,12.92,46.74,17.26170,0.0,1019.9


In [ ]:
#Clean Data
df = df.dropna()

# Create Features and Target
column_to_predict = "temp"
features = df.drop(columns=column_to_predict)
target = df[column_to_predict]
target

0        39.02
1        39.02
2        39.02
3        39.92
4        39.02
         ...  
26110    35.96
26111    33.98
26112    32.00
26113    30.92
26114    28.94
Name: temp, Length: 23383, dtype: float64

# Model library import

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Preparing training and test data, Grid search model

In [ ]:
# Hyperparameters to search for best MLP model
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate_init': [0.001, 0.01],
}

# Split the data into the training and testing variables
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
# Set the scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Perform the grid search
grid_search = GridSearchCV(MLPRegressor(max_iter=500, random_state=42), param_grid, scoring='neg_mean_squared_error', n_jobs=-8, cv=3, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)


# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error: {rmse}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


Root Mean Squared Error: 0.7098005147533872
Root Mean Squared Error (RMSE): 0.7098005147533872
Mean Absolute Error (MAE): 0.48681030424495025
R-squared: 0.9984515865106524


# Testing model with data point

In [ ]:
# Row to test model against
row = 2
sample_data = X_test_scaled[row]  # Retrive data from test
print(sample_data)

predicted_value = best_model.predict([sample_data])
print(f"Predicted {column_to_predict}: {predicted_value[0]}")  # Print the predicted column

year          2013.00000
month            1.00000
day             11.00000
hour            15.00000
dewp            39.92000
humid           82.38000
wind_speed       4.60312
precip           0.00000
pressure      1024.90000
Name: 253, dtype: float64
Actual Temperature:  44.96
Predicted Temperature: 44.858628799554545


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


# Model exporter for  use on weather app

In [ ]:
import joblib

# Save the model to disk
filename = 'finalized_model.sav'
joblib.dump(best_model, filename)

# Save Scaler
joblib.dump(scaler, 'scaler.sav')

# Save Features so that the order is the same on the app
feature_names = X_train.columns.tolist()
joblib.dump(feature_names, 'feature_names.sav')